In [2]:
import pandas as pd

In [2]:
df1 = pd.read_csv(r"C:\Users\ercj\Desktop\plain_language_summarization_platform\final_abstracts\0218_abstracts.csv")
df2 = pd.read_csv(r"C:\Users\ercj\Desktop\plain_language_summarization_platform\edited_personalizing_dataset.csv")
print(df1.columns)
print(df2.columns)

Index(['abstract_id', 'topic', 'method', 'link', 'human_written_pls',
       'abstract_title', 'abstract', 'question_1', 'question_2', 'question_3',
       'question_4', 'question_5', 'question_1_answers_choices',
       'question_1_correct_answers', 'question_2_answers_choices',
       'question_2_correct_answers', 'question_3_answers_choices',
       'question_3_correct_answers', 'question_4_answers_choices',
       'question_4_correct_answers', 'question_5_answers_choices',
       'question_5_correct_answers', 'chosen_questions', 'terms_chosen'],
      dtype='object')
Index(['abstract_id', 'topic', 'method', 'link', 'human_written_pls',
       'abstract_title', 'abstract', 'main_idea_question', 'method_question',
       'result_question', 'q1_question', 'q2_question', 'q3_question',
       'q4_question', 'q5_question', 'q1_level', 'q2_level', 'q3_level',
       'q4_level', 'q5_level', 'q1_option1_answer', 'q1_option2_answer',
       'q1_option3_answer', 'q2_option1_answer', 'q2_opti

In [4]:

question3_bank = [
    {
        "question": "Select all of the following that are fruits",
        "answers": "dog; blue; grape",
        "correct": "grape"
    },
    {
        "question": "Select all of the following that are colors",
        "answers": "blue; green; pink",
        "correct": "blue; green; pink"
    },
    {
        "question": "Select all of the following that are colors",
        "answers": "blue; green; pink",
        "correct": "blue; green; pink"
    },
    {
        "question": "Select the items that are animals",
        "answers": "Chair; Lamp; Dog",
        "correct": "Dog"
    }
]

In [ ]:
def process_dataframes(df1, df2, question3_bank):
    # Create a mapping for df2 for faster lookup
    df2_lookup = df2.set_index('abstract_id')
    
    for idx, row in df1.iterrows():
        abs_id = row['abstract_id']
        
        # 1. Get chosen questions (e.g., "5,6,8,9")
        if pd.isna(row['chosen_questions']):
            continue
            
        chosen_ids = [str(x).strip() for x in str(row['chosen_questions']).split(',')]
        target_slots = ['1', '2', '4', '5']
        
        for i, q_num in enumerate(chosen_ids[:4]):
            slot = target_slots[i]
            q_text = df2_lookup.loc[abs_id, f'q{q_num}_question']
            
            options = [
                str(df2_lookup.loc[abs_id, f'q{q_num}_option{j}_answer']) 
                for j in range(1, 4)
            ]
            options_str = "; ".join(options)
            
            # Find correct answers based on the "TRUE" flags
            correct_list = []
            for j in range(1, 4):
                is_correct = df2_lookup.loc[abs_id, f'q{q_num}_option{j}_correct']
                if str(is_correct).upper() == 'TRUE':
                    correct_list.append(options[j-1])
            correct_str = "; ".join(correct_list)
            
            # Update df1
            df1.at[idx, f'question_{slot}'] = q_text
            df1.at[idx, f'question_{slot}_answers_choices'] = options_str
            df1.at[idx, f'question_{slot}_correct_answers'] = correct_str

        bank_item = question3_bank[idx % len(question3_bank)]
        df1.at[idx, 'question_3'] = bank_item['question']
        df1.at[idx, 'question_3_answers_choices'] = bank_item['answers']
        df1.at[idx, 'question_3_correct_answers'] = bank_item['correct']

    return df1

df1_updated = process_dataframes(df1, df2, question3_bank)

In [ ]:
df1_updated.to_csv("final_combined_abstracts.csv")

: 